# EDA(ExploratoryDataAnalysis)

Reference:
- https://www.kaggle.com/jpmiller/basic-eda-with-images
- https://www.kaggle.com/jtlowery/plots-and-interactive-graphs-eda

## Run name

In [1]:
import time

project_name = 'inclusive-images-challenge'
step_name = 'EDA'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: inclusive-images-challenge_EDA_20181006-162037


## Important params

## Import PKGs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display
import seaborn as sns

import os
import sys
import gc
import math
import tqdm
import shutil
import zipfile
import pickle
import h5py
# import cv2
from PIL import Image

from tqdm import tqdm
import multiprocessing

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import shuffle

random_num = np.random.randint(10000)
cpu_amount = multiprocessing.cpu_count()

print('cpu_amount: %s' % (cpu_amount - 1))
print('random_num: %s' % random_num)

C:\Users\study\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


cpu_amount: 3
random_num: 7180


## Project folders

In [3]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')
stage_1_test_images_folder = os.path.join(input_folder, 'stage_1_test_images')

print(input_folder)
print(stage_1_test_images_folder)

class_descriptions_csv = os.path.join(input_folder, 'class-descriptions.csv')
class_trainable_csv = os.path.join(input_folder, 'classes-trainable.csv')
stage_1_attributions_csv = os.path.join(input_folder, 'stage_1_attributions.csv')
stage_1_sample_submission_csv = os.path.join(input_folder, 'stage_1_sample_submission.csv')
train_bounding_boxes_csv = os.path.join(input_folder, 'train_bounding_boxes.csv')
train_human_labels_csv = os.path.join(input_folder, 'train_human_labels.csv')
train_machine_labels_csv = os.path.join(input_folder, 'train_machine_labels.csv')
tuning_labels_csv = os.path.join(input_folder, 'tuning_labels.csv')

print(class_descriptions_csv)
print(class_trainable_csv)
print(stage_1_attributions_csv)
print(stage_1_sample_submission_csv)
print(train_bounding_boxes_csv)
print(train_human_labels_csv)
print(train_machine_labels_csv)
print(tuning_labels_csv)

D:\kaggle-Inclusive-Images\input
D:\kaggle-Inclusive-Images\input\stage_1_test_images
D:\kaggle-Inclusive-Images\input\class-descriptions.csv
D:\kaggle-Inclusive-Images\input\classes-trainable.csv
D:\kaggle-Inclusive-Images\input\stage_1_attributions.csv
D:\kaggle-Inclusive-Images\input\stage_1_sample_submission.csv
D:\kaggle-Inclusive-Images\input\train_bounding_boxes.csv
D:\kaggle-Inclusive-Images\input\train_human_labels.csv
D:\kaggle-Inclusive-Images\input\train_machine_labels.csv
D:\kaggle-Inclusive-Images\input\tuning_labels.csv


## Preview data

In [4]:
class_descriptions_df = pd.read_csv(class_descriptions_csv)
print(class_descriptions_df.shape)
display(class_descriptions_df.head(2))
print(len(set(class_descriptions_df['label_code'])), len(set(class_descriptions_df['description'])), 'Not equal --> One label_code may have many descriptions.')

(19987, 2)


,label_code,description
0,/m/0100nhbf,Sprenger's tulip
1,/m/0104x9kv,Vinegret


19987 19704 Not equal --> One label_code may have many descriptions.


In [5]:
class_trainable_df = pd.read_csv(class_trainable_csv)
print(class_trainable_df.shape)
display(class_trainable_df.head(2))

(7178, 1)


,label_code
0,/m/0108_09c
1,/m/010dmf


In [6]:
stage_1_attributions_df = pd.read_csv(stage_1_attributions_csv)
print(stage_1_attributions_df.shape)
display(stage_1_attributions_df.head(2))

(32580, 2)


,image_id,source
0,2b2b327132556c767a736b3d,Nita Ar
1,2b2b394755692f303963553d,Sathya Esarapu


In [7]:
stage_1_sample_submission_df = pd.read_csv(stage_1_sample_submission_csv)
print(stage_1_sample_submission_df.shape)
display(stage_1_sample_submission_df.head(2))

(32580, 2)


,image_id,labels
0,2b2b327132556c767a736b3d,/m/0sgh53y /m/0g4cd0
1,2b2b394755692f303963553d,/m/0sgh53y /m/0g4cd0


In [10]:
class_trainable_df = pd.read_csv(train_bounding_boxes_csv)
print(class_trainable_df.shape)
display(class_trainable_df.head(2))

(14610229, 13)


,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.0125,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.0250,0.276563,0.714063,0.948438,0,1,0,0,0
